In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 12:32:59.228136: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 12:32:59.339602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 12:32:59.339622: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 12:32:59.360936: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 12:32:59.896466: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

2022-11-07 12:33:04.675754: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:33:04.676295: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:33:04.676341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:33:04.676380: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 12:33:04.676418: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

Epoch 1/100
1/1 [==============================] - 0s 455ms/step - loss: 4.1496 - auc: 0.2114
Epoch 2/100
1/1 [==============================] - 0s 22ms/step - loss: 2.2806 - auc: 0.4778
Epoch 3/100
1/1 [==============================] - 0s 20ms/step - loss: 1.6924 - auc: 0.8428
Epoch 4/100
1/1 [==============================] - 0s 23ms/step - loss: 1.8187 - auc: 0.8728
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.9612 - auc: 0.8814
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 2.0390 - auc: 0.8818
Epoch 7/100
1/1 [==============================] - 0s 35ms/step - loss: 2.1005 - auc: 0.8791
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 2.0428 - auc: 0.8821
Epoch 9/100
1/1 [==============================] - 0s 47ms/step - loss: 1.9765 - auc: 0.8913
Epoch 10/100
1/1 [==============================] - 0s 42ms/step - loss: 1.8627 - auc: 0.8977
Epoch 11/100
1/1 [==============================] - 0s 34ms/step - l

1/1 [==============================] - 0s 41ms/step - loss: 0.2522 - auc: 0.9917
Epoch 89/100
1/1 [==============================] - 0s 38ms/step - loss: 0.2581 - auc: 0.9902
Epoch 90/100
1/1 [==============================] - 0s 43ms/step - loss: 0.2555 - auc: 0.9904
Epoch 91/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2424 - auc: 0.9919
Epoch 92/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2536 - auc: 0.9899
Epoch 93/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2454 - auc: 0.9908
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2521 - auc: 0.9895
Epoch 95/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2417 - auc: 0.9908
Epoch 96/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2480 - auc: 0.9906
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2388 - auc: 0.9920
Epoch 98/100
1/1 [==============================] - 0s 22ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 421ms/step - loss: 2.1206 - auc: 0.4661
Epoch 2/100
1/1 [==============================] - 0s 20ms/step - loss: 1.6854 - auc: 0.8046
Epoch 3/100
1/1 [==============================] - 0s 19ms/step - loss: 1.7647 - auc: 0.8375
Epoch 4/100
1/1 [==============================] - 0s 21ms/step - loss: 1.7965 - auc: 0.8494
Epoch 5/100
1/1 [==============================] - 0s 20ms/step - loss: 1.7037 - auc: 0.8607
Epoch 6/100
1/1 [==============================] - 0s 22ms/step - loss: 1.5395 - auc: 0.8739
Epoch 7/100
1/1 [==============================] - 0s 21ms/step - loss: 1.4038 - auc: 0.8758
Epoch 8/100
1/1 [==============================] - 0s 20ms/step - loss: 1.2613 - auc: 0.8894
Epoch 9/100
1/1 [==============================] - 0s 23ms/step - loss: 1.2231 - auc: 0.8826
Epoch 10/100
1/1 [==============================] - 0s 23ms/step - loss: 1.2150 - auc: 0.8661
Epoch 11/100
1/1 [==============================] - 0s 23ms/step - l

1/1 [==============================] - 0s 32ms/step - loss: 0.2381 - auc: 0.9903
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2482 - auc: 0.9884
Epoch 90/100
1/1 [==============================] - 0s 39ms/step - loss: 0.2439 - auc: 0.9888
Epoch 91/100
1/1 [==============================] - 0s 45ms/step - loss: 0.2398 - auc: 0.9899
Epoch 92/100
1/1 [==============================] - 0s 42ms/step - loss: 0.2535 - auc: 0.9869
Epoch 93/100
1/1 [==============================] - 0s 52ms/step - loss: 0.2482 - auc: 0.9880
Epoch 94/100
1/1 [==============================] - 0s 37ms/step - loss: 0.2541 - auc: 0.9867
Epoch 95/100
1/1 [==============================] - 0s 41ms/step - loss: 0.2456 - auc: 0.9881
Epoch 96/100
1/1 [==============================] - 0s 41ms/step - loss: 0.2347 - auc: 0.9908
Epoch 97/100
1/1 [==============================] - 0s 55ms/step - loss: 0.2451 - auc: 0.9884
Epoch 98/100
1/1 [==============================] - 0s 31ms/step - loss: 

In [5]:
m='remove_Connectivity'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 505ms/step - loss: 2.2483 - auc: 0.4231
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7148 - auc: 0.7971
Epoch 3/100
1/1 [==============================] - 0s 20ms/step - loss: 1.7676 - auc: 0.8437
Epoch 4/100
1/1 [==============================] - 0s 20ms/step - loss: 1.7918 - auc: 0.8570
Epoch 5/100
1/1 [==============================] - 0s 21ms/step - loss: 1.7524 - auc: 0.8632
Epoch 6/100
1/1 [==============================] - 0s 21ms/step - loss: 1.5844 - auc: 0.8842
Epoch 7/100
1/1 [==============================] - 0s 23ms/step - loss: 1.4256 - auc: 0.8925
Epoch 8/100
1/1 [==============================] - 0s 40ms/step - loss: 1.2900 - auc: 0.8997
Epoch 9/100
1/1 [==============================] - 0s 38ms/step - loss: 1.1857 - auc: 0.9141
Epoch 10/100
1/1 [==============================] - 0s 40ms/step - loss: 1.1793 - auc: 0.8989
Epoch 11/100
1/1 [==============================] - 0s 23ms/step - l

1/1 [==============================] - 0s 22ms/step - loss: 0.2253 - auc: 0.9908
Epoch 89/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2195 - auc: 0.9922
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2228 - auc: 0.9914
Epoch 91/100
1/1 [==============================] - 0s 37ms/step - loss: 0.2081 - auc: 0.9940
Epoch 92/100
1/1 [==============================] - 0s 42ms/step - loss: 0.2320 - auc: 0.9903
Epoch 93/100
1/1 [==============================] - 0s 33ms/step - loss: 0.2055 - auc: 0.9944
Epoch 94/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2200 - auc: 0.9923
Epoch 95/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2236 - auc: 0.9906
Epoch 96/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2118 - auc: 0.9932
Epoch 97/100
1/1 [==============================] - 0s 19ms/step - loss: 0.2166 - auc: 0.9922
Epoch 98/100
1/1 [==============================] - 0s 20ms/step - loss: 

In [6]:
m='remove_Charge'
a=['AtomPairFP','Connectivity',
       'Kappa','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 424ms/step - loss: 2.4182 - auc: 0.3560
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.7681 - auc: 0.7657
Epoch 3/100
1/1 [==============================] - 0s 22ms/step - loss: 1.7981 - auc: 0.8329
Epoch 4/100
1/1 [==============================] - 0s 24ms/step - loss: 1.8533 - auc: 0.8553
Epoch 5/100
1/1 [==============================] - 0s 26ms/step - loss: 1.8181 - auc: 0.8667
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7267 - auc: 0.8743
Epoch 7/100
1/1 [==============================] - 0s 44ms/step - loss: 1.5418 - auc: 0.8907
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3930 - auc: 0.9006
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2733 - auc: 0.9039
Epoch 10/100
1/1 [==============================] - 0s 21ms/step - loss: 1.1560 - auc: 0.9193
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - l

1/1 [==============================] - 0s 21ms/step - loss: 0.2182 - auc: 0.9927
Epoch 89/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2095 - auc: 0.9937
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2261 - auc: 0.9907
Epoch 91/100
1/1 [==============================] - 0s 38ms/step - loss: 0.2208 - auc: 0.9914
Epoch 92/100
1/1 [==============================] - 0s 34ms/step - loss: 0.2248 - auc: 0.9909
Epoch 93/100
1/1 [==============================] - 0s 38ms/step - loss: 0.2229 - auc: 0.9904
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2210 - auc: 0.9915
Epoch 95/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2186 - auc: 0.9914
Epoch 96/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2220 - auc: 0.9908
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2142 - auc: 0.9927
Epoch 98/100
1/1 [==============================] - 0s 21ms/step - loss: 

In [7]:
m='remove_Kappa'
a=['AtomPairFP','Connectivity',
       'Charge','MAP4','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 364ms/step - loss: 2.1613 - auc: 0.4729
Epoch 2/100
1/1 [==============================] - 0s 38ms/step - loss: 1.6458 - auc: 0.8206
Epoch 3/100
1/1 [==============================] - 0s 38ms/step - loss: 1.7364 - auc: 0.8529
Epoch 4/100
1/1 [==============================] - 0s 42ms/step - loss: 1.7556 - auc: 0.8703
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6687 - auc: 0.8807
Epoch 6/100
1/1 [==============================] - 0s 22ms/step - loss: 1.5183 - auc: 0.8955
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3703 - auc: 0.9058
Epoch 8/100
1/1 [==============================] - 0s 20ms/step - loss: 1.2332 - auc: 0.9155
Epoch 9/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1655 - auc: 0.9200
Epoch 10/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1309 - auc: 0.9184
Epoch 11/100
1/1 [==============================] - 0s 19ms/step - l

1/1 [==============================] - 0s 20ms/step - loss: 0.2207 - auc: 0.9911
Epoch 89/100
1/1 [==============================] - 0s 19ms/step - loss: 0.2141 - auc: 0.9930
Epoch 90/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2273 - auc: 0.9898
Epoch 91/100
1/1 [==============================] - 0s 19ms/step - loss: 0.2176 - auc: 0.9915
Epoch 92/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2233 - auc: 0.9902
Epoch 93/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2234 - auc: 0.9905
Epoch 94/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2120 - auc: 0.9926
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2110 - auc: 0.9927
Epoch 96/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2242 - auc: 0.9905
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2209 - auc: 0.9911
Epoch 98/100
1/1 [==============================] - 0s 20ms/step - loss: 

In [8]:
m='remove_MAP4'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MOE','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 348ms/step - loss: 3.5556 - auc: 0.2508
Epoch 2/100
1/1 [==============================] - 0s 19ms/step - loss: 1.9742 - auc: 0.5711
Epoch 3/100
1/1 [==============================] - 0s 21ms/step - loss: 1.5316 - auc: 0.8619
Epoch 4/100
1/1 [==============================] - 0s 21ms/step - loss: 1.6699 - auc: 0.8776
Epoch 5/100
1/1 [==============================] - 0s 20ms/step - loss: 1.8188 - auc: 0.8797
Epoch 6/100
1/1 [==============================] - 0s 19ms/step - loss: 1.8934 - auc: 0.8821
Epoch 7/100
1/1 [==============================] - 0s 21ms/step - loss: 1.9002 - auc: 0.8774
Epoch 8/100
1/1 [==============================] - 0s 20ms/step - loss: 1.8988 - auc: 0.8770
Epoch 9/100
1/1 [==============================] - 0s 19ms/step - loss: 1.7517 - auc: 0.8945
Epoch 10/100
1/1 [==============================] - 0s 21ms/step - loss: 1.6312 - auc: 0.9011
Epoch 11/100
1/1 [==============================] - 0s 19ms/step - l

1/1 [==============================] - 0s 21ms/step - loss: 0.2825 - auc: 0.9850
Epoch 89/100
1/1 [==============================] - 0s 20ms/step - loss: 0.2815 - auc: 0.9854
Epoch 90/100
1/1 [==============================] - 0s 20ms/step - loss: 0.2763 - auc: 0.9863
Epoch 91/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2727 - auc: 0.9870
Epoch 92/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2788 - auc: 0.9857
Epoch 93/100
1/1 [==============================] - 0s 20ms/step - loss: 0.2807 - auc: 0.9846
Epoch 94/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2781 - auc: 0.9845
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2670 - auc: 0.9869
Epoch 96/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2715 - auc: 0.9863
Epoch 97/100
1/1 [==============================] - 0s 20ms/step - loss: 0.2815 - auc: 0.9842
Epoch 98/100
1/1 [==============================] - 0s 22ms/step - loss: 

In [9]:
m='remove_MOE'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MorganFP','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 423ms/step - loss: 1.8020 - auc: 0.7389
Epoch 2/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7776 - auc: 0.8304
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.5479 - auc: 0.8530
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4208 - auc: 0.8495
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3688 - auc: 0.8430
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2725 - auc: 0.8790
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1610 - auc: 0.9203
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.0828 - auc: 0.9418
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 1.0285 - auc: 0.9518
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 0.9892 - auc: 0.9550
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2300 - auc: 0.9902
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2182 - auc: 0.9919
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2148 - auc: 0.9927
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2242 - auc: 0.9908
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2163 - auc: 0.9927
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2286 - auc: 0.9902
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2159 - auc: 0.9920
Epoch 95/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2220 - auc: 0.9912
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2192 - auc: 0.9920
Epoch 97/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2280 - auc: 0.9900
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [10]:
m='remove_MorganFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','Topology','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 2.0663 - auc: 0.5157
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 1.7203 - auc: 0.8139
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7957 - auc: 0.8405
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7433 - auc: 0.8595
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.6402 - auc: 0.8623
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4597 - auc: 0.8757
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3338 - auc: 0.8822
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2637 - auc: 0.8810
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2209 - auc: 0.8806
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2001 - auc: 0.8763
Epoch 11/100
1/1 [==============================] - 0s 35ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2370 - auc: 0.9902
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2409 - auc: 0.9901
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2371 - auc: 0.9902
Epoch 91/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2376 - auc: 0.9904
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2360 - auc: 0.9904
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2300 - auc: 0.9917
Epoch 94/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2275 - auc: 0.9915
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2329 - auc: 0.9902
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2413 - auc: 0.9885
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2304 - auc: 0.9912
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [11]:
m='remove_Topology'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP','TorsionFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 517ms/step - loss: 1.9118 - auc: 0.7660
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.7099 - auc: 0.7463
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5557 - auc: 0.8058
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4371 - auc: 0.8575
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3241 - auc: 0.8898
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2099 - auc: 0.9193
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1367 - auc: 0.9334
Epoch 8/100
1/1 [==============================] - 0s 25ms/step - loss: 1.0462 - auc: 0.9530
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9694 - auc: 0.9674
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 0.9263 - auc: 0.9711
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.2044 - auc: 0.9926
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1994 - auc: 0.9929
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2104 - auc: 0.9913
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2083 - auc: 0.9921
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2053 - auc: 0.9927
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2073 - auc: 0.9923
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2021 - auc: 0.9925
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2009 - auc: 0.9930
Epoch 96/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2120 - auc: 0.9911
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2005 - auc: 0.9929
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [12]:
m='remove_TorsionFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 469ms/step - loss: 1.9725 - auc: 0.7722
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7468 - auc: 0.7344
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6532 - auc: 0.7113
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4954 - auc: 0.7945
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3837 - auc: 0.8545
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2896 - auc: 0.8895
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2400 - auc: 0.8975
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1330 - auc: 0.9170
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.0478 - auc: 0.9330
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 0.9786 - auc: 0.9469
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2308 - auc: 0.9912
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2310 - auc: 0.9909
Epoch 90/100
1/1 [==============================] - 0s 37ms/step - loss: 0.2291 - auc: 0.9912
Epoch 91/100
1/1 [==============================] - 0s 47ms/step - loss: 0.2333 - auc: 0.9903
Epoch 92/100
1/1 [==============================] - 0s 46ms/step - loss: 0.2262 - auc: 0.9915
Epoch 93/100
1/1 [==============================] - 0s 38ms/step - loss: 0.2232 - auc: 0.9925
Epoch 94/100
1/1 [==============================] - 0s 45ms/step - loss: 0.2312 - auc: 0.9904
Epoch 95/100
1/1 [==============================] - 0s 45ms/step - loss: 0.2282 - auc: 0.9912
Epoch 96/100
1/1 [==============================] - 0s 44ms/step - loss: 0.2220 - auc: 0.9919
Epoch 97/100
1/1 [==============================] - 0s 47ms/step - loss: 0.2288 - auc: 0.9913
Epoch 98/100
1/1 [==============================] - 0s 31ms/step - loss: 

In [13]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('9feature_remove_one3.csv',index=False)
print(de)

                 model       auc
0            all_model  0.944907
1    remove_AtomPairFP  0.930196
2  remove_Connectivity  0.945542
3        remove_Charge  0.944251
4         remove_Kappa  0.945124
5          remove_MAP4  0.943476
6           remove_MOE  0.944982
7      remove_MorganFP  0.945077
8      remove_Topology  0.946570
9     remove_TorsionFP  0.948161
